In [12]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 14: Disk Defragmentation ---</h2><p>Suddenly, a scheduled job activates the system's <a href="https://en.wikipedia.org/wiki/Defragmentation">disk defragmenter</a>. Were the situation different, you might <a href="https://www.youtube.com/watch?v=kPv1gQ5Rs8A&amp;t=37">sit and watch it for a while</a>, but today, you just don't have that kind of time. It's soaking up valuable system resources that are needed elsewhere, and so the only option is to help it finish its task as soon as possible.</p>
<p>The disk in question consists of a 128x128 grid; each square of the grid is either <em>free</em> or <em>used</em>. On this disk, the state of the grid is tracked by the bits in a sequence of <a href="10">knot hashes</a>.</p>
<p>A total of 128 knot hashes are calculated, each corresponding to a single row in the grid; each hash contains 128 bits which correspond to individual grid squares. Each bit of a hash indicates whether that square is <em>free</em> (<code>0</code>) or <em>used</em> (<code>1</code>).</p>
<p>The hash inputs are a key string (your puzzle input), a dash, and a number from <code>0</code> to <code>127</code> corresponding to the row.  For example, if your key string were <code>flqrgnkx</code>, then the first row would be given by the bits of the knot hash of <code>flqrgnkx-0</code>, the second row from the bits of the knot hash of <code>flqrgnkx-1</code>, and so on until the last row, <code>flqrgnkx-127</code>.</p>
<p>The output of a knot hash is traditionally represented by 32 hexadecimal digits; each of these digits correspond to 4 bits, for a total of <code>4 * 32 = 128</code> bits. To convert to bits, turn each hexadecimal digit to its equivalent binary value, high-bit first: <code>0</code> becomes <code>0000</code>, <code>1</code> becomes <code>0001</code>, <code>e</code> becomes <code>1110</code>, <code>f</code> becomes <code>1111</code>, and so on; a hash that begins with <code>a0c2017...</code> in hexadecimal would begin with <code>10100000110000100000000101110000...</code> in binary.</p>
<p>Continuing this process, the <em>first 8 rows and columns</em> for key <code>flqrgnkx</code> appear as follows, using <code>#</code> to denote used squares, and <code>.</code> to denote free ones:</p>
<pre><code>##.#.#..--&gt;
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.--&gt;
|      |   
V      V   
</code></pre>
<p>In this example, <code>8108</code> squares are used across the entire 128x128 grid.</p>
<p>Given your actual key string, <em>how many squares are used</em>?</p>
</article>


<link href="style.css" rel="stylesheet"></link>


In [13]:
example = "flqrgnkx"

In [14]:
from functools import reduce
from itertools import batched
from operator import xor

from tabulate import tabulate


def reverse(p: int, l: int, n: int, lst: int) -> list[int]:
    lst1 = lst[p : p + l]
    if p + l >= n:
        lst1 += lst[: p + l - n]
        lst1 = lst1[::-1]
        lst = lst1[n - p :] + lst[p + l - n : p] + lst1[: n - p]
    else:
        lst = lst[:p] + lst1[::-1] + lst[p + l :]
    return lst


def knot_hash(lengths: str, n: int = 256) -> str:
    p, skip_size = 0, 0
    lengths = list(bytes(lengths, "ascii")) + [17, 31, 73, 47, 23]
    lst = list(range(n))

    for _ in range(64):
        for l in lengths:
            lst = reverse(p, l, n, lst)
            p = (p + l + skip_size) % n
            skip_size += 1

    return "".join(f"{reduce(xor, b[1:], b[0]):02x}" for b in batched(lst, 16))


def harddisk_state(s: str) -> list[str]:
    return [
        "".join(f"{int(b, base=16):04b}" for b in knot_hash(f"{s}-{i}"))
        for i in range(128)
    ]


def count_used(s: str) -> int:
    state = harddisk_state(s)
    return sum(sum(1 for b in l if b == "1") for l in state)


print(f"Example: {count_used(example)} should 8108")

Example: 8108 should 8108


In [15]:
puzzle = "stpzcrnm"
print(f"Part I: {count_used(puzzle)}")

Part I: 8250


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>8250</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now, <span title="This is exactly how it works in real life.">all the defragmenter needs to know</span> is the number of <em>regions</em>. A region is a group of <em>used</em> squares that are all <em>adjacent</em>, not including diagonals. Every used square is in exactly one region: lone used squares form their own isolated regions, while several adjacent squares all count as a single region.</p>
<p>In the example above, the following nine regions are visible, each marked with a distinct digit:</p>
<pre><code>11.2.3..--&gt;
.1.2.3.4   
....5.6.   
7.8.55.9   
.88.5...   
88..5..8   
.8...8..   
88.8.88.--&gt;
|      |   
V      V   
</code></pre>
<p>Of particular interest is the region marked <code>8</code>; while it does not appear contiguous in this small view, all of the squares marked <code>8</code> are connected when considering the whole 128x128 grid. In total, in this example, <code>1242</code> regions are present.</p>
<p><em>How many regions</em> are present given your key string?</p>
</article>

</main>


In [16]:
from collections import deque


def count_regions(s: str) -> int:
    state = harddisk_state(s)
    seen = set()
    groups = 0
    for r, c in product(range(128), repeat=2):
        if state[r][c] == "0" or (r, c) in seen:
            continue

        groups += 1

        queue = deque([(r, c)])
        while queue:
            rr, cc = queue.popleft()
            seen.add((rr, cc))
            queue.extend(
                (rr + dr, cc + dc)
                for dr, dc in ((-1, 0), (0, 1), (1, 0), (0, -1))
                if 0 <= rr + dr < 128
                and 0 <= cc + dc < 128
                and state[rr + dr][cc + dc] == "1"
                and (rr + dr, cc + dc) not in seen
            )

    return groups


print(f"Example: {count_regions(example)} should 1242")

Example: 1242 should 1242


In [17]:
print(f"Part II: {count_regions(puzzle)}")

Part II: 1113


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1113</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


<link href="style.css" rel="stylesheet"></link>
